# Simulate and Evaluate using AI as Judge Quality Evaluators with Azure AI Evaluation SDK

## Objective

This tutorial provides a step-by-step guide on how to evaluate prompts against variety of model endpoints deployed on Azure AI Platform or non Azure AI platforms. 

This guide uses Python Class as an application target which is passed to Evaluate API provided by Azure AI Evaluation SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [33]:
%pip install azure-ai-evaluation
%pip install promptflow-azure
%pip install azure-identity
%pip install --upgrade openai
#%pip install marshmallow==3.23.3

### Parameters and imports

In [34]:
import pandas as pd
import os
import json
import importlib.resources as pkg_resources
import requests
from typing import Any, Dict, List, Optional
from pathlib import Path
from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import GroundednessEvaluator
from azure.ai.evaluation.simulator import Simulator
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider


In [35]:
os.environ["AZURE_SUBSCRIPTION_ID"] = "xxxxx"#This is the subscription ID for the Azure service
os.environ["AZURE_OPENAI_ENDPOINT"] = "xxxxx"#This is the endpoint for the Azure OpenAI service
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-05-01-preview"#This is the API version for the Azure OpenAI service
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-4o"#This is the deployment for the Azure OpenAI service
os.environ["AZURE_AI_FOUNDRY_RESOURCE_GROUP"] = "xxxxxx"#This is the resource group for AI Foundry
os.environ["AZURE_AI_FOUNDRY_PROJECT_NAME"] = "xxxxxx"#This is the project name for AI Foundry

project_scope = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_AI_FOUNDRY_RESOURCE_GROUP"),
    "project_name": os.environ.get("AZURE_AI_FOUNDRY_PROJECT_NAME"),
}
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

search_endpoint = "xxxxx"#This is the endpoint for the Azure search service
index_name = "xxxxx"#   This is the index name for the Azure search service
api_key = "xxxxx"#This is the API key for the Azure search service


Create the simulator

In [36]:
from azure.ai.evaluation.simulator import Simulator

simulator = Simulator(model_config=model_config)

Connect application end point to the simulator

In [37]:
from application_endpoint import ApplicationEndpoint
 

async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = latest_message.get("context", None)
    # call model end point
    model_endpoint = ApplicationEndpoint(model_config)
    response = model_endpoint(query, None)
    print(response)
    # we are formatting the response to follow the openAI chat protocol format
    formatted_response = {
        "content": response["response"],
        "role": "assistant",
        "context": context,
    }
    messages["messages"].append(formatted_response)
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}

Get the context from the AI Search

In [38]:
import requests

def generate_text_from_index(search_term: str) -> str:
    url = f"{search_endpoint}/indexes/{index_name}/docs/search?api-version=2024-07-01"
    headers = {"Content-Type": "application/json","Api-key": api_key}
    search_query = {"search": search_term, "top": 10}
    print(f'Search query is {search_query}')
    response = requests.post(url=url, headers=headers, data=json.dumps(search_query))

    text = ""
    print(f'Statis code is {response.status_code}')
    if response.status_code == 200:
        results = response.json()
        print(results)
        for result in results["value"]:
            text += result["content"]
    
    return text[:5000]



Specify a search term for document

In [39]:
search_term = "Responsible AI"
text = generate_text_from_index(search_term)


Call the simultor

In [40]:
outputs = await simulator(
    target=callback,
    text=text,
    num_queries=2,
    max_conversation_turns=2,
    tasks=[
        f"I want to learn more about Responsible AI",
        f"I want to know how to implement Responsible AI in my organization",
    ],
)

## Save the simulate data

In [41]:
simulated_output_file = Path("simulated_output.json")
with simulated_output_file.open("a") as f:
    json.dump(outputs, f)

In [42]:
eval_data_file = Path("simulated_eval_data.jsonl")
with eval_data_file.open("w") as file:
    for output in outputs:
        file.write(output.to_eval_qr_json_lines())

## Validate The result

We will use Evaluate API provided by Prompt Flow SDK. It requires a target Application or python Function, which handles a call to LLMs and retrieve responses. 

In the notebook, we will use an Application Target `ModelEndpoints` to get answers from multiple model endpoints against provided question aka prompts. 

This application target requires list of model endpoints and their authentication keys. For simplicity, we have provided them in the `env_var` variable which is passed into init() function of `ModelEndpoints`.


Please provide Azure AI Project details so that traces and eval results are pushing in the project in Azure AI Studio.

In [43]:
azure_ai_project = {
    "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
    "resource_group_name": os.environ["AZURE_AI_FOUNDRY_RESOURCE_GROUP"],
    "project_name": os.environ["AZURE_AI_FOUNDRY_PROJECT_NAME"],
}

## Data

Following code reads Json file "data.jsonl" which contains inputs to the Application Target function. It provides question, context and ground truth on each line. 

In [44]:
df = pd.read_json("simulated_eval_data.jsonl", lines=True)
print(df.head())

## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [45]:
import os

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

## Run the evaluation

The Following code runs Evaluate API and uses Content Safety, Relevance and Coherence Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which can route the calls to specific model endpoints using model name in conditional logic.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

In [46]:
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)
from application_endpoint import ApplicationEndpoint


content_safety_evaluator = ContentSafetyEvaluator(
    azure_ai_project=azure_ai_project, credential=DefaultAzureCredential()
)
relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)

path = str(pathlib.Path(pathlib.Path.cwd())) + "/simulated_eval_data.jsonl"

results = evaluate(
    evaluation_name="Eval-Run" + "-" + "GPT35-08012025",
    data=path,
    evaluators={
        "content_safety": content_safety_evaluator,
        "coherence": coherence_evaluator,
        "relevance": relevance_evaluator,
        "groundedness": groundedness_evaluator,
        "fluency": fluency_evaluator,
        # "similarity": similarity_evaluator,
    },
    azure_ai_project=azure_ai_project,
    evaluator_config={
        "content_safety": {"column_mapping": {"query": "${data.query}", "response": "${data.response}"}},
        "coherence": {"column_mapping": {"response": "${data.response}", "query": "${data.query}"}},
        "relevance": {
            "column_mapping": {"response": "${data.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "groundedness": {
            "column_mapping": {
                "response": "${data.response}",
                "context": "${data.context}",
                "query": "${data.query}",
            }
        },
        "fluency": {
            "column_mapping": {"response": "${data.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        # "similarity": {
        #     "column_mapping": {"response": "${data.response}", "context": "${data.context}", "query": "${data.query}"}
        # },
    },
)

View the results

In [47]:
pprint(results)

In [48]:
pd.DataFrame(results["rows"])

,inputs.query,inputs.response,inputs.context,outputs.coherence.coherence,outputs.coherence.gpt_coherence,outputs.coherence.coherence_reason,outputs.relevance.relevance,outputs.relevance.gpt_relevance,outputs.relevance.relevance_reason,outputs.groundedness.groundedness,outputs.groundedness.gpt_groundedness,outputs.groundedness.groundedness_reason,outputs.fluency.fluency,outputs.fluency.gpt_fluency,outputs.fluency.fluency_reason,line_number
0,I'm curious about the guidelines for Responsib...,The title of the document that outlines Micros...,None,4,4,The RESPONSE is coherent and effectively addre...,4,4,The RESPONSE accurately and completely provide...,1,1,The RESPONSE introduces information about Micr...,3,3,The response is clear and grammatically correc...,0
1,What are some key principles outlined in Micro...,Microsoft's Responsible AI Standard outlines s...,None,4,4,The RESPONSE is coherent and effectively addre...,5,5,The RESPONSE fully addresses the QUERY by list...,1,1,The RESPONSE is completely unrelated to any CO...,4,4,"The RESPONSE is well-articulated, with good gr...",1
2,I'm curious about the specific goals outlined ...,The General Requirements of the Responsible AI...,None,4,4,The RESPONSE is coherent as it directly answer...,5,5,The RESPONSE fully addresses the QUERY by prov...,1,1,The RESPONSE is entirely unrelated to any CONT...,4,4,The RESPONSE demonstrates proficient fluency w...,2
3,That's a comprehensive list of goals! Can you ...,When implementing Responsible AI in your organ...,None,4,4,The RESPONSE is coherent and effectively addre...,4,4,The RESPONSE fully addresses the QUERY by prov...,1,1,The RESPONSE introduces a topic about Responsi...,4,4,"The RESPONSE is well-articulated, with good co...",3
